In [1]:
import pandas as pd
import os
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

0.40


In [2]:
onto = get_ontology("pbn_t3_5.owl").load()

# Overview

In [3]:
debug = True
if debug:
    print("\n# Overview\n")
    for k in onto.classes():
        if len(k.instances()):
            print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")
    print("\n# Details\n")
    for k in onto.classes():
        K = str(k).split(".")[-1]
        if K not in ["Qid","PBNThing"]:
            print("## ",K," -- ",len(k.instances()), "instances.")
            for i in k.instances()[:2]: 
                I = str(i).split(".")[-1]
                if K == "Qid":
                    if len(i.has_Qid_alias) > 0:
                        print("### ",I," -> ",i.has_Qid_alias[0])
                    else:
                        print("### ",I)
                    for p in list(i.get_properties()):
                        P = str(p).split(".")[-1]
                        OBJ = "onto."+str(I)+"."+str(P)
                        if not P == "label":
                            print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))
                else:
                    print("### ",I," -> ",i.label[0])
                    for p in list(i.get_properties()):
                        P = str(p).split(".")[-1]
                        OBJ = "onto."+str(I)+"."+str(P)
                        if not P == "label":
                            print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))



# Overview

*  _PBNThing_  --  19156 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1408 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Category_  --  3 instances.
*  _BP_Theme_  --  10 instances.
*  _PBNCategory_  --  280 instances.

# Details

##  BenefReturn  --  1606 instances.
###  PBN__BenefReturn_0  ->  Early detection helps public health authorities implement timely inter

# Creating mardown files and excel

### Creating QIDs

### Creating the rest

In [5]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
CLASSES = []
NAMECLASSES = []
for k in onto.classes():
    K = str(k).split(".")[-1]
    if (K not in ["Qid","PBNThing"]) and len(k.instances()):
        ClassName = K
        print(K)
        NAMECLASSES.append(K)
        CLASS = {}
        PATH = "./docs/"+K+"/"
        #PATH = '7.pages/'+K+"/"
        if not os.path.isdir(PATH):
            os.mkdir(PATH)
        #print("## ",K," -- ",len(k.instances()), "instances.")
        for i in k.instances(): 
            try:
                I = str(i).split(".")[-1]
                CLASS[i] = {}
                CLASS[i]["ID"] = I
                MD = "[Home](https://github.com/mm80843/T3.5/blob/main/docs/index.md) >> Class: ["+K+"](https://github.com/mm80843/T3.5/tree/main/docs/"+K+"/index.md)"+" >> Individual ID:"+i.get_name()+" \n\n"
                MD += "# __"+i.label[0]+"__\n\n"
                FILENAME = PATH+i.get_name()+".md"
                A = list(i.get_properties())
                A.sort(key=lambda x: str(x))
                for p in A:
                    P = str(p).split(".")[-1]
                    OBJ = "onto."+str(I)+"."+str(P)
                    OB = eval(OBJ)
                    E = eval(OBJ)[0]
                    if not (OB == [onto.nan]):
                        if not P == "label":
                            if not p.label:
                                MD += "## Property: "+P+"\n\n"
                            else:
                                MD += "## "+str(p.label[0])+"\n\n"
                                

                            E = eval(OBJ)[0]

                                                        
                            if type(E) is str:
                                MD += E + "\n\n"
                                CLASS[i][P] = E
                            else:

                                A = [x.get_name() for x in eval(OBJ)]
                                if "_" in str(A[0]):
                                    A = [LINK+x.split("PBN__")[1].split("_")[0]+"/"+x+".md" for x in A]
                                    #print(str(A[0]),A)
                                else:                                
                                    A = [LINK+"/Qid/"+x+".md" for x in A]
                                #print(A,eval(OBJ))
                                if (OB[0].is_instance_of[0] == onto.Qid):
                                    B = [() for x in eval(OBJ)]
                                else:
                                    B = [x.label[0] for x in eval(OBJ)]
                                for k in range(len(A)):
                                    if not B[k] == "Nan":
                                        if not "-----" in B[k]:
                                            MD += "* ["+B[k]+"]("+A[k]+")\n"
                                MD += "\n"
                                CLASS[i][P] = [str(x) for x in B]

                            CLASS[i]["NAME"] = E
                            CLASS[i]["PBNTYPE"] = ClassName                                
                        else:
                            E = eval(OBJ)[0]
                            CLASS[i]["NAME"] = E
                            CLASS[i]["PBNTYPE"] = ClassName
                #print(MD)
                with open(FILENAME,"w",encoding="utf-8") as f:
                    f.write(MD)
            except:
                pass
        CLASSES.append(CLASS)

BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Technology
ISO_Impact
ISO_Purpose
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
BP_Category
BP_Theme
PBNCategory


In [6]:
str(i.is_instance_of[0]).split(".")[-1],ClassName

('PBNCategory', 'PBNCategory')

# Creating the excel of the KG

In [ ]:
ALLDFs = []
for  k in range(len(CLASSES)):
    D = pd.DataFrame(CLASSES[k]).T
    if len(D):
        D = D.convert_dtypes()
        C = list(D.columns)
        if len(C):
            DT = D.dtypes
            for x in range(len(DT)):
                if DT[x] == "object":
                    if C[x] not in [ "has_TechSource"	,"has_TechGroup",	"has_TechSubgroup","has_BenefInArticle"	,"has_RiskInArticle",	"has_SourceMitigation"	,"has_SourceTech","has_BenefitForPeople","has_Closest",  "has_SubjectRisk", "has_RiskOwner" , "has_RiskSource", "has_RiskHealth","has_RiskISO_Purpose","has_RiskISO_Impact","has_RiskType","has_RiskGBN"]:
                        D = D.explode(C[x])
            D = D.convert_dtypes()
            CC = [x for x in C if x not in ["ID","NAME","PBNTYPE"]]
            CC = ["PBNTYPE","ID","NAME"] + CC
            print(k,list(D.columns),list(CC))
            D = D[CC]
            ALLDFs.append(D)

0 ['ID', 'has_ReturnBenef', 'NAME', 'PBNTYPE'] ['PBNTYPE', 'ID', 'NAME', 'has_ReturnBenef']
1 ['ID', 'has_BenefDetails', 'NAME', 'PBNTYPE', 'has_BenefReturn', 'has_BenefSource', 'has_Beneficiary', 'has_BenefitMitigation'] ['PBNTYPE', 'ID', 'NAME', 'has_BenefDetails', 'has_BenefReturn', 'has_BenefSource', 'has_Beneficiary', 'has_BenefitMitigation']
2 ['ID', 'has_ArticleDOI', 'NAME', 'PBNTYPE', 'has_ArticleLLMShortSummary', 'has_ArticleLLMSummary', 'has_ArticleRef', 'has_ArticleTitle', 'has_ArticleYear', 'has_BenefInArticle', 'has_RiskInArticle', 'has_SourceMitigation', 'has_SourceTech'] ['PBNTYPE', 'ID', 'NAME', 'has_ArticleDOI', 'has_ArticleLLMShortSummary', 'has_ArticleLLMSummary', 'has_ArticleRef', 'has_ArticleTitle', 'has_ArticleYear', 'has_BenefInArticle', 'has_RiskInArticle', 'has_SourceMitigation', 'has_SourceTech']
3 ['ID', 'has_Category', 'NAME', 'PBNTYPE', 'has_Closest', 'has_RiskMitigation', 'has_RiskDescription', 'has_RiskHealth', 'has_RiskISOScale', 'has_RiskISO_Impact', 'h

In [ ]:
i = 0
for OneSheet in ALLDFs:
    title = OneSheet.iloc[0].PBNTYPE
    print(i,title,len(OneSheet))
    i += 1

In [ ]:
with pd.ExcelWriter("outputs/pbn_t3_5.xlsx") as writer:
    i = 0
    for OneSheet in ALLDFs:
        title = OneSheet.iloc[0].PBNTYPE
        print(i,title,len(OneSheet))
        i += 1
        OneSheet[:(1048576-1)].to_excel(writer, sheet_name=title, index=False, engine="xlsxwriter")
        print("... done.")


# Creating  index

### Creating general indexes

In [4]:
IDX = ""
IDX += "# Overview\n\n"
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
BB = list(onto.classes())
BB.sort(key=lambda x: str(x).lower())
for k in BB:
    PATH =  "./docs/"
    #PATH = "7.pages/" 
    if len(k.instances()):
        MM = str(k).split(".")[-1]
        if (not MM == "RiskMitigation") and (not MM == "PBNThing"):
            IDX += "* _["+MM+"]("+LINK+MM+"/index.md)_ -- "+str(len(k.instances()))+ " instances.\n"
with open(PATH+"index.md","w",encoding="utf-8") as f:
        f.write(IDX)

print(IDX)

# Overview

* _[Article](https://github.com/mm80843/T3.5/blob/main/docs/Article/index.md)_ -- 376 instances.
* _[Benef](https://github.com/mm80843/T3.5/blob/main/docs/Benef/index.md)_ -- 1408 instances.
* _[BenefReturn](https://github.com/mm80843/T3.5/blob/main/docs/BenefReturn/index.md)_ -- 1606 instances.
* _[Blueprint](https://github.com/mm80843/T3.5/blob/main/docs/Blueprint/index.md)_ -- 24 instances.
* _[BP_Category](https://github.com/mm80843/T3.5/blob/main/docs/BP_Category/index.md)_ -- 3 instances.
* _[BP_Enabler](https://github.com/mm80843/T3.5/blob/main/docs/BP_Enabler/index.md)_ -- 33 instances.
* _[BP_Intervention](https://github.com/mm80843/T3.5/blob/main/docs/BP_Intervention/index.md)_ -- 40 instances.
* _[BP_Permanent](https://github.com/mm80843/T3.5/blob/main/docs/BP_Permanent/index.md)_ -- 4 instances.
* _[BP_Phase](https://github.com/mm80843/T3.5/blob/main/docs/BP_Phase/index.md)_ -- 4 instances.
* _[BP_Scale](https://github.com/mm80843/T3.5/blob/main/docs/BP_Scale/in

### Creating classes indexes

In [5]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
for k in onto.classes():
    K = str(k).split(".")[-1]
    if len(k.instances()):
    #print(K)
        PATH =  "./docs/"
        #PATH = "7.pages/" 
        if K not in ["PBNThing","Qid"]:
            IDX = "[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)"
            IDX += "\n\n# "+str(K)+" -- "+str(len(k.instances()))+ " instances.\n\n"
            BB = list(k.instances())
            BB.sort(key=lambda x: x.label[0].lower())
            for i in BB: 
                I = str(i).split(".")[-1]
                if K == "Qid":
                    if len(i.has_Qid_alias) > 0:

                        IDX += "* ["+i.has_Qid_alias[0]+"]("+LINK+"Qid/"+str(I)+".md)\n"
                    else:
                        IDX += "* ["+str(I)+"]\n"
                else:
                    II = I.split("PBN__")[1].split("_")[0]
                    A = i.label[0]
                    if K == "Article":
                        A = " ".join(list(set(A.split(" ")))).replace("nan","").strip()
                    if "Nan" not in A:
                        if "----" not in A:
                            IDX += "* ["+A+"]("+LINK+II+"/"+str(I)+".md)\n" 
            with open(PATH+K+"/index.md","w",encoding="utf-8") as f:
                f.write(IDX)

print(IDX)

[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)

# PBNCategory -- 280 instances.

* [A collection of diverse entities devoted to various causes, including mental and physical health, business, research, conservation, and social welfare.](https://github.com/mm80843/T3.5/blob/main/docs/PBNCategory/PBN__PBNCategory_200.md)
* [A collective group, including marginalized and vulnerable, local and global communities, individuals, and communication entities, particularly in underserved areas.](https://github.com/mm80843/T3.5/blob/main/docs/PBNCategory/PBN__PBNCategory_193.md)
* [A collective of individuals and groups who contribute to and support the well-being of a community.](https://github.com/mm80843/T3.5/blob/main/docs/PBNCategory/PBN__PBNCategory_220.md)
* [A comprehensive array of digital platforms, resources, and professionals offering support, counseling, therapy, and interventions for mental health.](https://github.com/mm80843/T3.5/blob/main/docs/PBNCategory/PBN__PB

In [6]:
print("DONE")

DONE
